In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
class stock_bands():

    def __init__(self, ticker):
        before_ticker = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
        after_ticker = '?apikey=551e3f6296368a437bff1db55fa1af26'
        url = before_ticker + ticker + after_ticker
        data = requests.get(url).json()
        self.df = pd.DataFrame(data['historical'])
        self.df['date'] = pd.to_datetime(self.df['date'])
        
    def get_df(self):
        return self.df
    
    def add_bands(self, days, std):
        with_bands = self.df.copy()
        averages = []
        stdvs = []
        for i in range(len(with_bands)):
            if i+days >= len(with_bands):
                averages.append(np.nan)
                stdvs.append(np.nan)
            else:
                window = with_bands.loc[i : i + days - 1]
                closing_prices = window['close']
                averages.append(closing_prices.mean())
                stdvs.append(closing_prices.std())
        
        with_bands['MA'] = averages
        with_bands['STD'] = stdvs
        with_bands['Upper Bound'] = with_bands['MA'] + (std*with_bands['STD'])
        with_bands['Lower Bound'] = with_bands['MA'] - (std*with_bands['STD'])
        with_bands = with_bands.dropna()
        
        up_down = []
        curr_up_down = np.nan
        for j in range(len(with_bands)-1, -1, -1):
            if with_bands.loc[j,'close'] > with_bands.loc[j, 'Upper Bound']:
                curr_up_down = 'up'
            elif with_bands.loc[j,'close'] < with_bands.loc[j, 'Lower Bound']:
                curr_up_down = 'down'
            up_down.append(curr_up_down)
        up_down = up_down[::-1]
        with_bands['Up Down'] = up_down
        return with_bands
    
    def get_return(self, banded_df):
        pct_return = 1
        in_price = 0
        out_price = 0
        bought = False
        for i in range(len(banded_df)-1, -1, -1):
            if bought == False and banded_df.loc[i, 'Up Down'] == 'up':
                in_price = banded_df.loc[i, 'close']
                bought = True
            elif bought == True and banded_df.loc[i, 'Up Down'] == 'down':
                bought = False
                out_price = banded_df.loc[i, 'close']
                pct_return = pct_return*(1+(out_price - in_price)/in_price)
            elif bought == True and i == 0:
                out_price = banded_df.loc[i, 'close']
                pct_return = pct_return*(1+(out_price - in_price)/in_price)
        return pct_return - 1
    
    def get_best_params(self, low_day, high_day, low_std, high_std):
        max_return = 0
        optimal_days = 0
        optimal_std = 0
        for days in range(low_day, high_day):
            for std in np.arange(low_std, high_std, 0.1):
                test = apple.add_bands(days, std)
                test_return = apple.get_return(test)
                if test_return > max_return:
                    max_return = test_return
                    optimal_days = days
                    optimal_std = std
        output = 'Max Return: ' + str(max_return) + ' Optimal Days: ' + str(optimal_days)
        return  output + ' Optimal Standard Deviations: ' + str(optimal_std)